In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [34]:
# load datasets
review_text_test = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_test.csv")
review_text_train = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_text_train.csv")
review_text_test_meta = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_meta_test.csv")
review_text_train_meta = pd.read_csv("COMP30027_2020_assignment2_datasets\\review_meta_train.csv")

In [38]:
review_text_train

,review
0,dear longman & eagle.......you've left me no c...
1,Delish. The hubby and I wanted to do brunch on...
2,"yep, I've giving Yolk 5 stars. It's just reall..."
3,"Meat, meat, meat. It's meat-tastic. So much me..."
4,I caught up with the law school girls on a Sat...
...,...
28063,This afternoon I went to Yolk with my college ...
28064,Place has lots of side dishes. But that's abou...
28065,I am a huge fan of Brazillian steakhouses I've...
28066,Great Brunch.\n


In [37]:
review_text_train_meta

,date,review_id,reviewer_id,business_id,vote_funny,vote_cool,vote_useful,rating
0,3/15/2012,-s77HISu8DVQ8F0HxmWW6A,mthr7h15a_z9m9jRI6mG6Q,m5_GCJP2W4zEJnyVgxa3eA,0,1,3,1
1,6/21/2010,A2aCzGCgg6gAbatHiCrPfA,rhM01fl3iU0xHr3TIpCMhQ,m5_GCJP2W4zEJnyVgxa3eA,0,0,0,5
2,7/29/2011,DK2pd,SNHKDgmGiLn5chUlhdLCkg,CwPi6NVuJIZZx4IBcTekFQ,1,0,1,5
3,3/10/2006,b3BkUiWJEKNQko,HXjk1RVfLMPeZxitnk1Auw,43rd1LKcZRIunySzbMsyLQ,17,3,3,3
4,8/28/2012,RabHhte,W0ny0BqO0OJ4K4aVnSIlBw,CwPi6NVuJIZZx4IBcTekFQ,0,0,0,3
...,...,...,...,...,...,...,...,...
28063,5/8/2008,AoNuz5VBHjul-xWpp7Nbbg,JU96tV1CNyw5G03Yv79PRA,CwPi6NVuJIZZx4IBcTekFQ,0,0,0,1
28064,8/12/2012,F4b9FwPxyMRQO86eFiyYEA,IpVFQuWk4gEgQQ_XmzjeQQ,_n3OmwreEGPQmiHKklsD8w,0,0,0,1
28065,3/11/2010,D6Vel2u0lHi7QJK-jx,-vywj5I_Bh1FOzTe_sbQUw,43rd1LKcZRIunySzbMsyLQ,0,0,5,5
28066,3/9/2011,JSZrJ1yvndgXSKazIBkAOg,sjR61dvvQODav54vBNk0cg,dKcO9OQ44RPRlkWe-vToFA,0,0,0,5


In [68]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

text_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                    ('tfidf',TfidfTransformer()),
                    ('clf',DecisionTreeClassifier(max_depth = 20)),
                    ])

text_clf = text_clf.fit(review_text_train["review"],review_text_train_meta["rating"])

In [69]:
predicted = text_clf.predict(review_text_train["review"])
np.mean(predicted == review_text_train_meta["rating"])

0.857916488527861

In [74]:
from sklearn.model_selection import GridSearchCV
parameters = {
            'vect__ngram_range':[(1,1),(1,2)],
             'tfidf__use_idf':(True,False),
             'clf__max_depth':[8,10,12,14,16,18,20]
             }

In [75]:
gs_clf = GridSearchCV(text_clf,parameters,cv = 5,n_jobs = -1)
gs_clf = gs_clf.fit(review_text_train["review"],review_text_train_meta["rating"])

In [77]:
gs_clf.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='ignore',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabular...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=False)),
                ('clf',
                 DecisionTreeClassifier(class_weight=None, criterion='gini',
                                        max_depth=16, max_features=None,
                             

In [76]:
predicted = gs_clf.best_estimator_.predict(review_text_train["review"])
np.mean(predicted == review_text_train_meta["rating"])

0.8201154339461308

In [ ]:
from sklearn.neural_network import MLPClassifier


MLP_clf = Pipeline([('vect',CountVectorizer(stop_words="english",decode_error='ignore')),
                    ('tfidf',TfidfTransformer()),
                    ('clf',MLPClassifier()),
                    ])

parameters = {
            'vect__ngram_range':[(1,1),(1,2)],
             'tfidf__use_idf':(True,False),
             'clf__hidden_layer_sizes':[(i*20,)*i  for i in range(5,10)]
             'clf__alpha':[0.00001, 0.0001,0.001]
             }

gs_clf = GridSearchCV(MLP_clf,parameters,cv = 5,n_jobs = -1)
gs_clf = gs_clf.fit(review_text_train["review"],review_text_train_meta["rating"])

In [80]:
 [(i*20,)*i  for i in range(5,10) ]

[(100, 100, 100, 100, 100),
 (120, 120, 120, 120, 120, 120),
 (140, 140, 140, 140, 140, 140, 140),
 (160, 160, 160, 160, 160, 160, 160, 160),
 (180, 180, 180, 180, 180, 180, 180, 180, 180)]